In [2]:
%pip install llama-index 'google-generativeai>=0.3.0' matplotlib qdrant_client PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 733.2 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


##  Use Gemini to understand Images from URLs

In [4]:
import os

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

In [6]:
import google.generativeai as genai

genai.configure(
    api_key=GOOGLE_API_KEY,
    client_options={"api_endpoint": "generativelanguage.googleapis.com"},
)

## Read Restaurant Menu from pdf and store it in a Llama_index Document

In [22]:
   from llama_index import Document
   from PyPDF2 import PdfReader

   # Extract text from PDF
   with open("example-menu.pdf", "rb") as file:
       reader = PdfReader(file)
       text = ""
       for page in reader.pages:
           text += page.extract_text()

   # Create Document object
   document = Document(text=text, metadata={"source": "example-menu.pdf"})
   print(document)

Doc ID: ddf7ac6e-11fc-4445-b651-780aaee8d138
Text: Denotes LongHorn  Steakhouse® Specialty.LongHorn Steakhouse®,
LongHorn®, and all related trademarks, service marks, copyrights, and
indicia are owned by RARE Hospitality Management, LLC.  ©2020 RARE
Hospitality Management, LLC.  *Contains (or may contain) raw or
undercooked ingredients. Consuming raw or undercooked meats, poultry,
seafood, shell...


## Sore the Document in a qdrant Vector DB

In [23]:
from llama_index import VectorStoreIndex, StorageContext, ServiceContext
from llama_index.embeddings import GeminiEmbedding
from llama_index.llms import Gemini
from llama_index.vector_stores import QdrantVectorStore
from llama_index import StorageContext
import qdrant_client

# Create a local Qdrant vector store
client = qdrant_client.QdrantClient(path="vector_store")

vector_store = QdrantVectorStore(client=client, collection_name="collection")

# Using the embedding model to Gemini
embed_model = GeminiEmbedding(
    model_name="models/embedding-001", api_key=GOOGLE_API_KEY
)
service_context = ServiceContext.from_defaults(
    llm=Gemini(api_key=GOOGLE_API_KEY), embed_model=embed_model
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    [document],
    show_progress=True,
    service_context=service_context,
    storage_context=storage_context,
)

Generating embeddings: 100%|██████████| 5/5 [00:04<00:00,  1.09it/s]


### Using Gemini to synthesize the results and respond to queries

In [27]:
query_engine = index.as_query_engine(
    # similarity_top_k=1,
)

response = query_engine.query(
    "What are some information you can tell me about the restaurant?"
)
print(response)

- The restaurant offers a variety of steaks, including the Lean and hearty USDA Choice center-cut top sirloin, LongHorn® Steak Tips, Ribeye, New York Strip, and Fire-Grilled T-Bone.
- All entrées are served with a choice of side and hand-chopped salad.
- The restaurant also offers a variety of appetizers, including Seasoned Steakhouse Wings, Wild West Shrimp®, Spicy Chicken Bites, Texas Tonion®, White Cheddar Stuffed Mushrooms, and Firecracker Chicken Wraps.
- The restaurant has a full bar and offers a variety of cocktails, including The Perfect Margarita, Strawberry Margarita, Mango Margarita, Don Julio Margarita, LongHorn® Old Fashioned, Tito’s Texas Tea, Mules, Moscow Mules, and Sangrias.
